In [21]:
!pip install simpletransformers

In [107]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from simpletransformers.ner import NERModel,NERArgs
import re

In [23]:
df = pd.read_csv('/kaggle/input/fixed-tags-for-ner/fixed_csv_for_ner.csv').drop('Unnamed: 0', axis=1)

In [77]:

def transform_dataframe(df):
    # Преобразуем столбцы 'Text' и 'Tags' к строке, обрабатывая NaN
    df['Text'] = df['Text'].fillna('').astype(str)
    df['Tags'] = df['Tags'].fillna('').astype(str)

    # Создаем пустой DataFrame для результата
    result_df = pd.DataFrame(columns=['sentence_id', 'words', 'labels'])

    # Номер предложения
    sentence_id = 0

    # Итерация по строкам DataFrame
    for index, row in tqdm(df.iterrows()):
        text = row['Text'].split()  # разбиваем текст на слова
        tags = row['Tags'].split()  # разбиваем теги на список тегов

        # Убедимся, что количество слов соответствует количеству тегов
        if len(text) != len(tags):
            print(f"Warning: Mismatch at row {index} with text '{row['Text']}' and tags '{row['Tags']}'")
            continue  # Пропускаем текущую запись

        temp_df = pd.DataFrame({
            'sentence_id': [sentence_id] * len(text),
            'words': text,
            'labels': tags
        })

        # Добавляем временный DataFrame в итоговый
        result_df = pd.concat([result_df, temp_df], ignore_index=True)

        # Увеличиваем номер предложения
        sentence_id += 1

    return result_df


transformed_df = transform_dataframe(df)
transformed_df

13001it [06:58, 18.19it/s]

16572it [10:58, 25.17it/s]


,sentence_id,words,labels
0,0,фокус,"['O',"
1,0,недели,"'O',"
2,0,фн,"'O',"
3,0,сегодня,"'O',"
4,0,ммк,"'ORG',"
...,...,...,...
1367155,16570,марта,"'O',"
1367156,16570,м,"'O',"
1367157,16570,видео,"'O',"
1367158,16570,мсфо,"'O',"


In [25]:
label = transformed_df["labels"].unique().tolist()

args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [26]:
label

["['O',", "'O',", "'ORG',", "'O']", "['ORG',", "'ORG']", "['O']"]

In [27]:
model = NERModel('bert', 'ai-forever/ruBert-base',labels=label, args=args)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
X = transformed_df[['sentence_id', 'words']]
y = transformed_df['labels']

train_data, test_data = train_test_split(transformed_df, test_size=0.2)

In [29]:
model.train_model(train_data, eval_data = test_data, acc=accuracy_score)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/518 [00:00<?, ?it/s]

(518, 0.12672990950320678)

In [30]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/5 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/500 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 'O', seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 'O'] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 'ORG', seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ['O', seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ['ORG', seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.

In [31]:
result

{'eval_loss': 0.11346666055172681,
 'precision': 0.6566566058792198,
 'recall': 0.6735171696149844,
 'f1_score': 0.6649800300512413}

In [32]:
prediction, model_output = model.predict(["vk совершает сделку с yandex, после чего вк решили, что стоит и покупать дзен, а после этого руснефть приняла решение создать новые облегации"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
prediction

[[{'vk': "'ORG',"},
  {'совершает': "'O',"},
  {'сделку': "'O',"},
  {'с': "'O',"},
  {'yandex,': "'ORG',"},
  {'после': "'O',"},
  {'чего': "'O',"},
  {'вк': "'ORG',"},
  {'решили,': "'O',"},
  {'что': "'O',"},
  {'стоит': "'O',"},
  {'и': "'O',"},
  {'покупать': "'O',"},
  {'дзен,': "'O',"},
  {'а': "'O',"},
  {'после': "'O',"},
  {'этого': "'O',"},
  {'руснефть': "'ORG',"},
  {'приняла': "'O',"},
  {'решение': "'O',"},
  {'создать': "'O',"},
  {'новые': "'O',"},
  {'облегации': "'O',"}]]

In [34]:
model.save_model(output_dir='/kaggle/working/', model=model.model)

In [40]:
df_id = pd.read_excel('/kaggle/input/12345678/names and synonyms.xlsx')
df_id = df_id[['issuerid', 'Unnamed: 11']]

In [41]:
df_id

,issuerid,Unnamed: 11
0,1,Держава
1,2,Московский кредитный банк
2,3,Российский акционерный коммерческий дорожный банк
3,4,алроса
4,5,Авангард
...,...,...
250,270,Хендерсон
251,271,Sovcombank
252,272,АЗС Трасса
253,273,Делимобил


In [58]:
org_words = []
for sentence in prediction:
    for word_dict in sentence:
        for word, tag in word_dict.items():
            if "'ORG'," in tag:  # Проверяем, содержится ли тег 'ORG'
                org_words.append(word)  # Добавляем слово в список
org_words

['vk', 'yandex,', 'вк', 'руснефть']

In [109]:
def clean_text(text):
    return re.sub(r'[^\w\s]', '', text).lower()
org_words = [clean_text(word) for word in org_words]
org_words

['vk', 'yandex', 'вк', 'руснефть']

In [100]:
df_id = pd.read_excel('/kaggle/input/12345678/names and synonyms.xlsx')

In [103]:
fixed_ner_first = df_id['Unnamed: 11']
df_id['issuerid'] = df_id['issuerid'].apply(lambda x: str(x))
all_new = df_id[['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'issuerid']]
df_id = pd.concat([fixed_ner_first, all_new], axis=1)
df_id

,Unnamed: 11,Unnamed: 11,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 11,issuerid
0,Держава,Держава,Держава,DERZHAVA,DERZ,NaN,NaN,NaN,Держава,Держава,1
1,Московский кредитный банк,Московский кредитный банк,Московский кредитный банк,мкб,Credit Bank of Moscow,Credit Bank,NaN,NaN,Московский кредитный банк,Московский кредитный банк,2
2,Российский акционерный коммерческий дорожный банк,Российский акционерный коммерческий дорожный банк,Российский акционерный коммерческий дорожный банк,РДБанк,Дорожный банк,Russian public joint-stock commercial roads Bank,RosDorBank,РосДорБанк,Российский акционерный коммерческий дорожный банк,Российский акционерный коммерческий дорожный банк,3
3,алроса,алроса,алроса,alrosa,NaN,NaN,NaN,NaN,алроса,алроса,4
4,Авангард,Авангард,Авангард,AVANGARD,NaN,NaN,NaN,NaN,Авангард,Авангард,5
...,...,...,...,...,...,...,...,...,...,...,...
250,Хендерсон,Хендерсон,Хендерсон,NaN,NaN,NaN,NaN,NaN,Хендерсон,Хендерсон,270
251,Sovcombank,Sovcombank,Sovcombank,NaN,NaN,NaN,NaN,NaN,Sovcombank,Sovcombank,271
252,АЗС Трасса,АЗС Трасса,АЗС Трасса,"АЗС ""Трасса""",АЗС «Трасса»,NaN,NaN,NaN,АЗС Трасса,АЗС Трасса,272
253,Делимобил,Делимобил,Делимобил,Каршеринг Рус,delimobil,"ПАО ""Каршеринг Руссия""","""Каршеринг Руссия"", ПАО",NaN,Делимобил,Делимобил,273


In [ ]:
def remove_nans(values_list):
    return [value.lower() for value in values_list if not pd.isna(value)]

data = {}
for i in range(len(df_id.values)):
    data[df_id.values[i][-1]] = list(set(df_id.values[i]))

for key in data.keys():
    data[key] = remove_nans(data[key])

In [ ]:
data

In [110]:
org_keys = {word: set() for word in org_words}

for word in org_words:
    for key, names in data.items():
        if word in names:
            org_keys[word].add(key)

print(org_keys)

{'vk': {'223'}, 'yandex': {'236'}, 'вк': {'223'}, 'руснефть': set()}


In [72]:
df_id[df_id['issuerid'] == '236']

,Unnamed: 11,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,issuerid
235,Яндекс,Яндекс,Yandex,NaN,NaN,NaN,NaN,Яндекс,236
